## 📌 Extracción

In [ ]:

import pandas as pd
import requests

# URL del JSON en GitHub (versión RAW)
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# Descarga y carga de datos
resp = requests.get(url)
resp.raise_for_status()
data = resp.json()

# Si es un dict con clave 'data', úsala
if isinstance(data, dict):
    data = data.get('data', data)

df = pd.json_normalize(data)
df.head()


## 🔧 Transformación

In [ ]:

# Copia de trabajo
df_clean = df.copy()

# Normalizar nombres de columnas
df_clean.columns = [c.strip().lower().replace(" ", "_") for c in df_clean.columns]

# Conversión numérica donde sea posible
for col in df_clean.columns:
    if df_clean[col].dtype == object:
        try:
            df_clean[col] = pd.to_numeric(df_clean[col])
        except:
            pass

# Conversión de fechas si aplicable
for col in df_clean.columns:
    if any(k in col for k in ["date", "fecha", "start", "end"]):
        df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")

df_clean.head()


## 📊 Carga y análisis

In [ ]:

import matplotlib.pyplot as plt

# Resumen rápido
print("Filas:", len(df_clean))
print("Columnas:", df_clean.shape[1])
print(df_clean.describe(include='all'))

# Ejemplo: histograma de una variable numérica
num_cols = df_clean.select_dtypes(include='number').columns[:5]
for col in num_cols:
    df_clean[col].dropna().plot(kind='hist', bins=20, title=f"Histograma: {col}")
    plt.xlabel(col)
    plt.show()

# Guardar CSV limpio
df_clean.to_csv("telecomx_clean.csv", index=False)


## 📄 Informe final

In [ ]:

# Ejemplo de insights automáticos simples
nulos_prom = df_clean.isna().mean().mean() * 100
print(f"Promedio de valores nulos: {nulos_prom:.2f}%")

# Intentar detectar churn
target_col = [c for c in df_clean.columns if 'churn' in c or 'cancel' in c]
print("Posibles columnas objetivo:", target_col)

# Aquí puedes añadir análisis específicos y exportar resultados
df_clean.describe().to_csv("resumen_estadistico.csv")
